## Import necessary libraries

In [72]:
import yfinance as yf
import numpy as np
import pandas as pd

# Data collection

## Fetch daily OHLCV data 

In [73]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-26')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-26')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [74]:
print(tsla.tail())

Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2025-03-19  235.860001  241.410004  229.199997  231.610001  111993800
2025-03-20  236.259995  238.000000  230.050003  233.350006   99028300
2025-03-21  248.710007  249.520004  234.550003  234.990005  132728700
2025-03-24  278.390015  278.640015  256.329987  258.079987  169079900
2025-03-25  288.140015  288.200012  271.279999  283.600006  150361500


In [75]:
print(xly.head())

Price           Close       High        Low       Open   Volume
Ticker            XLY        XLY        XLY        XLY      XLY
Date                                                           
2019-01-02  94.046097  94.574020  91.246207  91.745849  6840800
2019-01-03  92.009811  93.555877  91.849550  93.329626  6346000
2019-01-04  95.054817  95.752431  93.065670  93.254212  7269100
2019-01-07  97.204224  97.826417  95.460187  95.507318  6263100
2019-01-08  98.278931  98.806854  96.883697  98.212940  9391000


## Fetching sentiment scores from the API

In [76]:
import requests
import pandas as pd

# Step 1: Make the API request
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date_from=20250101T0130&date_to=20250301T0130&limit=1000&tickers=tsla&apikey=A6UVECU631A2QQFG'
response = requests.get(url)

# Step 2: Parse the JSON
if response.status_code == 200:
    sentiment_json = response.json()
    articles = sentiment_json['feed']
    sentiment_df = pd.DataFrame(articles)
else:
    raise Exception("API call failed:", response.status_code)

# Step 3: Extract time and sentiment score
# Not all rows have sentiment data, so we need to drop those
sentiment_df = sentiment_df.dropna(subset=['ticker_sentiment'])

# Flatten the nested 'ticker_sentiment' column (which is a list of dicts)
# We'll explode it, then extract the sentiment_score
sentiment_df = sentiment_df.explode('ticker_sentiment')
sentiment_df['sentiment_score'] = sentiment_df['ticker_sentiment'].apply(lambda x: float(x['ticker_sentiment_score']) if isinstance(x, dict) else None)

# Step 4: Convert timestamp to date only
sentiment_df['date'] = pd.to_datetime(sentiment_df['time_published'], format="%Y%m%dT%H%M%S").dt.date

# Step 5: Group by date and compute average sentiment score
daily_sentiment = sentiment_df.groupby('date')['sentiment_score'].mean().to_frame(name='avg_sentiment_score')

# Done! You now have a clean DataFrame
print(daily_sentiment.tail())
len(daily_sentiment)

            avg_sentiment_score
date                           
2025-03-25             0.074306
2025-03-26             0.091014
2025-03-27             0.053512
2025-03-28             0.083595
2025-03-29            -0.227385


25

# Indicator Calculation

## Compute VI+ and VI- for TSLA

In [77]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [78]:
vi_plus, vi_minus = calculate_vortex(tsla, "TSLA")
vortex_tsla = pd.DataFrame({
    "VI+": vi_plus,
    "VI-": vi_minus
})

vortex_tsla.tail(5)

,VI+,VI-
Date,,
2025-03-19,0.676217,1.075796
2025-03-20,0.691925,1.079572
2025-03-21,0.680821,1.077461
2025-03-24,0.805184,0.911496
2025-03-25,0.843336,0.858793


## Calculate Volume Weighted Sentiment

In [79]:
# Step 2: Ensure datetime index for sentiment data
# If daily_sentiment index is of type `date`, convert to datetime
daily_sentiment.index = pd.to_datetime(daily_sentiment.index)

# Step 3: Extract volume from tsla
volume_series = tsla["Volume"]
volume_series.index = pd.to_datetime(volume_series.index)

# Step 4: Merge everything by date
combined_df = vortex_tsla.join(daily_sentiment, how="left")
volume_series.name = "Volume"
combined_df = combined_df.join(volume_series, how="left")


combined_df = combined_df.dropna()

# Show result
print(combined_df.tail())

len(daily_sentiment)

                 VI+       VI-  avg_sentiment_score       TSLA
Date                                                          
2025-03-19  0.676217  1.075796             0.100433  111993800
2025-03-20  0.691925  1.079572             0.081449   99028300
2025-03-21  0.680821  1.077461             0.078977  132728700
2025-03-24  0.805184  0.911496             0.092694  169079900
2025-03-25  0.843336  0.858793             0.074306  150361500


25

In [80]:
# Rename 'TSLA' column to 'Volume'
combined_df = combined_df.rename(columns={"TSLA": "Volume"})

# Create a new column: volume-weighted sentiment
combined_df["volume_weighted_sentiment"] = combined_df["avg_sentiment_score"] * combined_df["Volume"]

# Drop any rows with missing data (just in case)
combined_df = combined_df.dropna()

# Show result
print(combined_df.tail())

                 VI+       VI-  avg_sentiment_score     Volume  \
Date                                                             
2025-03-19  0.676217  1.075796             0.100433  111993800   
2025-03-20  0.691925  1.079572             0.081449   99028300   
2025-03-21  0.680821  1.077461             0.078977  132728700   
2025-03-24  0.805184  0.911496             0.092694  169079900   
2025-03-25  0.843336  0.858793             0.074306  150361500   

            volume_weighted_sentiment  
Date                                   
2025-03-19               1.124792e+07  
2025-03-20               8.065711e+06  
2025-03-21               1.048249e+07  
2025-03-24               1.567263e+07  
2025-03-25               1.117283e+07  


## Derive ATR for Volatility Adjustments

In [81]:
high = tsla["High"]
low = tsla["Low"]
close = tsla["Close"]

# True Range (TR)
tr = pd.concat([
    high - low,
    abs(high - close.shift(1)),
    abs(low - close.shift(1))
], axis=1).max(axis=1)

# ATR(10)
atr_10 = tr.rolling(window=10).mean()

# Optional: Add to your combined_df
combined_df["ATR_10"] = atr_10
print(combined_df.tail())

                 VI+       VI-  avg_sentiment_score     Volume  \
Date                                                             
2025-03-19  0.676217  1.075796             0.100433  111993800   
2025-03-20  0.691925  1.079572             0.081449   99028300   
2025-03-21  0.680821  1.077461             0.078977  132728700   
2025-03-24  0.805184  0.911496             0.092694  169079900   
2025-03-25  0.843336  0.858793             0.074306  150361500   

            volume_weighted_sentiment     ATR_10  
Date                                              
2025-03-19               1.124792e+07  19.417001  
2025-03-20               8.065711e+06  18.303999  
2025-03-21               1.048249e+07  18.248999  
2025-03-24               1.567263e+07  16.974998  
2025-03-25               1.117283e+07  16.663000  


# Signal Generation

In [82]:
# Step 1: Compute crossovers
vi_cross_up = (combined_df["VI+"].shift(1) < combined_df["VI-"].shift(1)) & (combined_df["VI+"] > combined_df["VI-"])
vi_cross_down = (combined_df["VI+"].shift(1) > combined_df["VI-"].shift(1)) & (combined_df["VI+"] < combined_df["VI-"])

# Step 2: Compute sentiment condition
combined_df["vw_sentiment_5d_avg"] = combined_df["volume_weighted_sentiment"].rolling(window=5).mean()
sentiment_positive = combined_df["vw_sentiment_5d_avg"] > 0

# Step 3: Compute ATR condition (in % of price)
# Use Close price from tsla DataFrame
price = tsla["Close"]
price.index = pd.to_datetime(price.index)
combined_df["price"] = price
combined_df["atr_pct"] = combined_df["ATR_10"] / combined_df["price"] * 100

# Step 4: Choose your strategy: Aggressive (ATR < 3) or Conservative (ATR ≥ 3)
# Let’s say we use both for now
aggressive_entry = combined_df["atr_pct"] < 3
conservative_entry = combined_df["atr_pct"] >= 3

# Step 5: Combine conditions
buy_signal = vi_cross_up & sentiment_positive & (aggressive_entry | conservative_entry)
sell_signal = vi_cross_down  # We'll handle trailing stop in backtest logic

# Step 6: Add to DataFrame
combined_df["signal"] = 0
combined_df.loc[buy_signal, "signal"] = 1
combined_df.loc[sell_signal, "signal"] = -1

print(combined_df["signal"].value_counts())

combined_df.head(5)

signal
0    15
Name: count, dtype: int64


,VI+,VI-,avg_sentiment_score,Volume,volume_weighted_sentiment,ATR_10,vw_sentiment_5d_avg,price,atr_pct,signal
Date,,,,,,,,,,
2025-03-05,0.736173,1.221730,0.108449,94042900,1.019882e+07,20.471997,NaN,279.100006,7.335004,0
2025-03-06,0.650565,1.297453,0.066316,98451600,6.528928e+06,20.950000,NaN,263.450012,7.952173,0
2025-03-07,0.598733,1.333346,0.072116,102369600,7.382454e+06,20.446001,NaN,262.670013,7.783911,0
2025-03-10,0.500406,1.297775,0.032752,189076900,6.192651e+06,22.943004,NaN,222.149994,10.327709,0
2025-03-11,0.488991,1.372778,0.009234,174896400,1.615057e+06,21.619003,6.383582e+06,230.580002,9.375923,0


In [83]:
import vectorbt as vbt

# Entry and exit signals
entries = combined_df["signal"] == 1
exits = combined_df["signal"] == -1

# Price series
price = combined_df["price"]

portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001,
    sl_stop=0.03  # <-- this is the trailing stop-loss!
)


In [84]:
# Plot equity curve
portfolio.plot().show()

ValueError: invalid unit abbreviation: B

In [ ]:
# Full stats
portfolio.stats()


/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2025-02-04 00:00:00
End                           2025-03-25 00:00:00
Period                                         35
Start Value                              100000.0
End Value                                100000.0
Total Return [%]                              0.0
Benchmark Return [%]                   -26.534249
Max Gross Exposure [%]                        0.0
Total Fees Paid                               0.0
Max Drawdown [%]                              NaN
Max Drawdown Duration                         NaN
Total Trades                                    0
Total Closed Trades                             0
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [ ]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046112,94.574035,91.246222,91.745864,6840800
2019-01-03,92.009842,93.555908,91.849581,93.329657,6346000
2019-01-04,95.054817,95.752431,93.065670,93.254212,7269100
2019-01-07,97.204208,97.826402,95.460172,95.507303,6263100
2019-01-08,98.278908,98.806831,96.883674,98.212917,9391000


In [ ]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [ ]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954697,227.889090,223.117390,223.144604,126925200,NaN,NaN
2019-01-03,221.538956,225.494199,221.049078,225.185752,144140700,NaN,NaN
2019-01-04,228.959625,229.612786,224.224218,224.605226,142628800,NaN,NaN
2019-01-07,230.764877,232.189121,228.324599,229.231765,103139100,NaN,NaN
2019-01-08,232.932968,233.422845,230.420129,232.978343,102512600,NaN,NaN
2019-01-09,234.021591,234.874329,232.406837,233.649650,95006600,NaN,NaN
2019-01-10,234.847137,235.101143,231.780912,232.470367,96823900,NaN,NaN
2019-01-11,234.937836,234.965050,233.168851,233.758503,73858100,NaN,NaN


                                               title  \
0  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   
1  New Inflation Data Dismays Bulls, CoreWeave IP...   
2  Assessing Apple's Performance Against Competit...   
3  Apple's Options Frenzy: What You Need to Know ...   
4  Trading The SPY As PCE Inflation Report Takes ...   

                                                 url   time_published  \
0  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   
1  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
2  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   
3  https://www.benzinga.com/insights/options/25/0...  20250328T134533   
4  https://www.benzinga.com/markets/equities/25/0...  20250328T125805   

               authors                                            summary  \
0        [Chris Katje]  Benzinga readers pick their favorite Magnifice...   
1   [The Arora Report]  To gain an edge, this is what you need to know

In [ ]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,EXCLUSIVE: Which Magnificent 7 Stock Will Perf...,https://www.benzinga.com/tech/25/03/44544129/e...,20250328T190544,[Chris Katje],Benzinga readers pick their favorite Magnifice...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.266216,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1984..."
1,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
2,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
3,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
4,Trading The SPY As PCE Inflation Report Takes ...,https://www.benzinga.com/markets/equities/25/0...,20250328T125805,[RIPS],Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.084859,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0813..."
5,EU To Issue Minimal Fines To Apple And Meta To...,https://www.benzinga.com/markets/25/03/4453457...,20250328T122729,[Namrata Sen],The European Union ( EU ) is reportedly set to...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.103777,Neutral,"[{'ticker': 'META', 'relevance_score': '0.2658..."
6,3 Northern Mutual Funds for Solid Returns,https://www.zacks.com/stock/news/2436613/3-nor...,20250328T111100,[Zacks Equity Research],"Invest in Northern mutual funds like NOIEX, NM...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.272886,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1496..."
7,2 Breakout Growth Stocks You Can Buy and Hold ...,https://www.fool.com/investing/2025/03/28/2-br...,20250328T110700,[Anders Bylund],Looking for undervalued stocks with explosive ...,https://g.foolcdn.com/editorial/images/812691/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.298788,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.1478..."
8,Intel's SuperFluid Cooling Tech 'Possibly Suit...,https://www.benzinga.com/25/03/44532855/intels...,20250328T104410,[Rishabh Mishra],Intel Corp. INTC SuperFluid cooling tech could...,https://editorial-assets.benzinga.com/wp-conte...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.226797,Somewhat-Bullish,"[{'ticker': 'NVDA', 'relevance_score': '0.4078..."
9,Should Strive 500 ETF ( STRV ) Be on Your In...,https://www.zacks.com/stock/news/2436589/shoul...,20250328T102007,[Zacks Equity Research],Style Box ETF report for ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic

## volatility

In [ ]:

# Flatten MultiIndex columns 
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate True Range
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#tsla["allocation_dollars"] = tsla["position_size"] * capital

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [ ]:
import plotly.express as px
fig = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [ ]:
import plotly.express as px

# Filter only 2025 data
tsla_2025 = tsla[tsla.index.year == 2025]

# Plot
fig = px.line(tsla_2025, x=tsla_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI+_"],
    mode='lines',
    name='VI+_',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI-_"],
    mode='lines',
    name='VI-_',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for TSLA",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()


In [ ]:
tsla_2025 = tsla.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI+_"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI-_"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for TSLA - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for SPY",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [ ]:
spy_2025 = spy.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for SPY - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for XLY",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [ ]:
xly_2025 = xly.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for XLY - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("SPY - 2025", "XLY - 2025", "TSLA - 2025")
)

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI+"],
    name="VI+ (SPY)",
    line=dict(color='blue'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI-"],
    name="VI- (SPY)",
    line=dict(color='orange'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI+"],
    name="VI+ (XLY)",
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI-"],
    name="VI- (XLY)",
    line=dict(color='orange'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI+_"],
    name="VI+ (TSLA)",
    line=dict(color='blue'),
    showlegend=False
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI-_"],
    name="VI- (TSLA)",
    line=dict(color='orange'),
    showlegend=False
), row=3, col=1)

fig.update_layout(
    height=500, width=1200,
    title_text="Vortex Indicator (VI+ and VI−) - 2025 Comparison",
    template="plotly_white"
)

fig.show()

In [ ]:
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("SPY Year To Year", "XLY Year To Year", "TSLA Year To Year")
)

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI+"],
    name="VI+ (SPY)",
    line=dict(color='blue'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI-"],
    name="VI- (SPY)",
    line=dict(color='orange'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI+"],
    name="VI+ (XLY)",
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI-"],
    name="VI- (XLY)",
    line=dict(color='orange'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI+_"],
    name="VI+ (TSLA)",
    line=dict(color='blue'),
    showlegend=False
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI-_"],
    name="VI- (TSLA)",
    line=dict(color='orange'),
    showlegend=False
), row=3, col=1)

fig.update_layout(
    height=900, width=1200,
    title_text="Vortex Indicator (VI+ and VI−) - Full Period Comparison",
    template="plotly_white"
)

fig.show()
